<a href="https://colab.research.google.com/github/yun890503/titanic-analysis_feature/blob/main/Untitled19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np
# 1. 請問資料中有哪些「欄位」你覺得是多餘的，在分析過程中應該被排除的呢？為什麼？
    #PassengerId：乘客ID，通常是用來標識乘客的唯一值，對生存與否沒有實際意義。

    # Name：乘客姓名，通常也不具有直接的影響。

    # Ticket：船票號碼，對於生存與否沒有太大的關聯。

    # Cabin：客艙號碼，缺失值較多，而且與生存與否的關聯度較低。

    # 這些欄位可能在分析中不會提供有用的資訊，因此可以考慮在分析過程中將它們排除。

# 2. 針對「有缺失值」的欄位，嘗試不同的處理策略（例如：常數、中位數、平均數填補）比較結果

# # 讀取資料集
df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')
# 找出缺失值
df.isnull().sum()# age,cabin,embarked三列含有缺失值
df_constant=df.copy()
from sklearn.impute import SimpleImputer

# 找出有缺失值的欄位
columns_with_missing = df.columns[df.isnull().any()].tolist()
# 中位數填補
constant_imputer = SimpleImputer(strategy='constant', fill_value='Missing')
df_constant[columns_with_missing] = constant_imputer.fit_transform(df_constant[columns_with_missing])
df_constant[columns_with_missing]
# ---------------------------------------------------------------------------------------------------
# 平均值填補
df_constant1=df.copy()
# 指定要填補的特定列（欄位）
subset_columns = ['Age']
# 用平均數填補
mean_imputer = SimpleImputer(strategy='mean')
df_constant1[subset_columns] = mean_imputer.fit_transform(df_constant1[subset_columns])
df_constant1[subset_columns]
# ---------------------------------------------------------------------------------------------------
#最常出現的值填補
df_constant1 = df.copy()
from sklearn.impute import SimpleImputer

# 找出有缺失值的欄位
columns_with_missing = df_constant1.columns[df.isnull().any()].tolist()
# 指定要填補的特定列（欄位）
subset_columns = ['Age']
# 使用最常出現的值填補文字或類別型的欄位
mode_imputer = SimpleImputer(strategy='most_frequent')
df_constant1[subset_columns] = mode_imputer.fit_transform(df_constant1[subset_columns])


# 3. 針對「非數值型」的欄位，嘗試不同的處理策略（例如：LabelEncoder 或 OneHotEncoder）比較結果。
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# 複製資料集
df_encoded = df.copy()

# 選取要轉換的欄位（列）
columns_to_encode = ['Sex', 'Embarked']

# 使用 LabelEncoder 將非數值型欄位轉換成數值
label_encoder = LabelEncoder()
for column in columns_to_encode:
    df_encoded[column] = label_encoder.fit_transform(df_encoded[column])

# 使用 OneHotEncoder 將非數值型欄位進行獨熱編碼
onehot_encoder = OneHotEncoder(sparse=False)
encoded_data = onehot_encoder.fit_transform(df_encoded[columns_to_encode])

# 建立新的 DataFrame 來儲存編碼後的結果
encoded_df_label = pd.DataFrame(df_encoded, columns=df.columns)
encoded_df_onehot = pd.DataFrame(encoded_data, columns=onehot_encoder.get_feature_names_out(columns_to_encode))

# 合併編碼後的 DataFrame
df_encoded = pd.concat([encoded_df_label, encoded_df_onehot], axis=1)

# 移除原始非數值型欄位（列）
df_encoded.drop(columns=columns_to_encode, inplace=True)

# 顯示結果
print(df_encoded)


# 4. 請你嘗試不同的特徵工程手法，產生可以幫助分析結果的新變數。

import pandas as pd
import numpy as np

#讀取資料集
df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

data = pd.DataFrame(df)

#資料前處理
data.info()
data.describe()
data.isnull().sum()

#將缺失值填補
data['Age'].fillna(data['Age'].mean() , inplace = True)
data['Embarked'].fillna(data['Embarked'].mode()[0] , inplace = True)
data['Fare'].fillna(data['Fare'].mean() , inplace = True)
data['Cabin'].fillna('None' , inplace = True)
data.isnull().sum()

#新增特徵
data['Family_Size'] = data['SibSp'] + data['Parch'] + 1
data['IsAlone'] = 0
data.loc[data['Family_Size'] == 1 , 'IsAlone'] = 1
data['Title'] = data['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
data['FareBin'] = pd.qcut(data['Fare'] , 4)
data['AgeBin'] = pd.cut(data['Age'].astype(int) , 5)

#刪除不需要的欄位
data.drop(['Cabin'] , axis = 1 , inplace = True)
data.drop(['PassengerId' , 'Name' , 'Ticket'] , axis = 1 , inplace = True)
data.head()
data.info()
data.describe()
data.isnull().sum()

#將資料集命名為df_train，準備模型訓練
df_train = data
df_train.head()
df_train.info()
df_train.describe()
df_train.isnull().sum()

#定義特徵欄位和目標欄位
#使用Label Encoding 進行特徵工程
from sklearn.preprocessing import LabelEncoder
#將特徵欄位進行編碼
columns_to_encode = ['Survived' , 'Age' , 'Embarked' , 'Family_Size' , 'Title' , 'FareBin' , 'AgeBin' , 'Pclass' , 'Sex','SibSp' ,'Parch' , 'IsAlone' ]
le = LabelEncoder()
le.fit(columns_to_encode)
#print(le.classes_)

#對需要編碼的特徵欄位逐個進行 LabelEncoder 編碼
for col in columns_to_encode[1:]:
  le = LabelEncoder()
  df_train[col] = le.fit_transform(df_train[col])

#檢查結果
#print(df_train[columns_to_encode])

df_train.head()
#定義特徵欄位和目標欄位
columns_X = list(df_train.columns)
columns_X.remove('Survived')
columns_y = ['Survived']
train_X = df_train[columns_X]
train_y = df_train[columns_y]
#使用Logistic 迴歸模型進行交叉驗證
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')

print(scores)







/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name   Age  SibSp  Parch  \
0                              Braund, Mr. Owen Harris  22.0      1      0   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0      1      0   
2                               Heikkinen, Miss. Laina  26.0      0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0      1      0   
4                             Allen, Mr. William Henry  35.0      0      0   
..                                                 ...   ...    ...    ... 